# xgboost

In [ ]:
import xgboost as xgb
train = np.array(shirase_spc.loc[:, ['relative_wind_y', 'swh']])
test = np.array(shirase_spc.loc[:, 'label'])

dtrain = xgb.DMatrix(train, label=test)
params = {'objective': 'binary:logistic', 'silent':1, 'random_state':71}
num_round = 50
model = xgb.train(params, dtrain, num_round)

fscore = model.get_score(importance_type='total_gain')
fscore = sorted([(k, v) for k , v in fscore.items()], key=lambda tpl: tpl[1], reverse=True)
print('xgboost importance')
print(fscore[:5])

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# データ読み込み
X_train, X_test = train_test_split(train)
y_train, y_test = train_test_split(test)

# xgboostモデルの作成
clf = xgb.XGBClassifier()

# ハイパーパラメータ探索
clf_cv = GridSearchCV(clf, {'max_depth': 2, 'n_estimators': 50}, verbose=1)
clf_cv.fit(X_train, y_train)
print (clf_cv.best_params_, clf_cv.best_score_)

# 改めて最適パラメータで学習
clf = xgb.XGBClassifier(**clf_cv.best_params_)
clf.fit(X_train, y_train)

# 学習モデルの保存、読み込み
# import pickle
# pickle.dump(clf, open("model.pkl", "wb"))
# clf = pickle.load(open("model.pkl", "rb"))

# 学習モデルの評価
pred = clf.predict(X_test)
print (confusion_matrix(y_test, pred))
print (classification_report(y_test, pred))

# PCA

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import os

def plot_reduced_graph(data, label, image_name, model='tsne',
                                     random_sampling=False, random_state=0,
                                     file_path='/Users/nakamurataiki/Desktop/sprayf/dataset/cruise/jare60'):
    if random_sampling:
        data = data.sample(1000, random_state=random_state)
        label = label.sample(1000, random_state=random_state)
        
    data = np.array(data)
    label = np.array(label)
        
    if model == 'pca':
        pca = PCA(n_components=2, random_state=random_state)
        transformed = pca.fit_transform(data)
    elif model == 'tsne':
        transformed = TSNE(n_components=2, random_state=random_state).fit_transform(data)
    
    transformed_1 = transformed[label == 1]
    transformed_0 = transformed[label == 0]
    
    plt.figure(figsize=(7, 7))
    plt.scatter(transformed_0[:, 0], transformed_0[:, 1], color='black', label='0', alpha=0.1)
    plt.scatter(transformed_1[:, 0], transformed_1[:, 1], color='yellow', label='1')
    plt.title("PCA: FFT")
    plt.show()
    #plt.savefig(os.path.join(file_path, image_name+'.png'))

reduction_data = []
#reduction_data = ['swh', '速度 (kt)', '相対風速 (m/s)', 0, 1]
#reduction_data = ['Relative wind speed[m/s]', 'swh']
reduction_data.extend([i for i in range(1, 31)])
for_reduction = without_tp.loc[:, reduction_data]
label = without_tp.loc[:, 'label']
print(for_reduction.shape, label.shape)
plot_reduced_graph(for_reduction, label, 'pca_relativewind_swh', model='tsne', random_sampling=True)